In [1]:
import requests
import pandas as pd
import numpy as np
from typing import List
from numpy.linalg import inv

In [2]:
# Model 1  y = a*SL + b*SW + c*PL + d*PW + e
def my_model_1() -> List[float]:
    # Insert a column of ones
    vectorones = y*0+1
    df.insert(6,'ones', vectorones, True)
    
    # Put the 5 columns into an array
    olddata = np.asarray(df[['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'ones']]).T
    
    ### Create the coefficient matrix A and load vector v, find the coef
    A = np.zeros((5,5))
    v = np.zeros((5,1))
    
    for i in range(5):
        v[i] = sum(olddata[i] * y.T[0])
        for j in range(5):
            A[i][j] = sum(olddata[i]*olddata[j])
    
    return np.matmul(inv(A),v)

In [3]:
# Classify one data record
def my_model_1_classify(newdata: List[float]) -> str:
    myindicator = sum(sum(mycoef.T*[newdata[0], newdata[1], newdata[2], newdata[3], 1]))
    
    if myindicator < 15:
        return "Iris-setosa"
    else:
        if myindicator < 25:
            return "Iris-versicolor"
        else:
            return "Iris-virginica"


In [4]:
# Calculate the accuracy 
def my_model_1_accuracy() -> float:
    my_counter_total = 0.0
    my_counter_right = 0.0
    datalist = np.asarray(df[['sepal-length', 'sepal-width', 'petal-length', 'petal-width']])
    for i in range(len(oldclass)):
        my_counter_total += 1.0
        if my_model_1_classify(datalist[i]) == oldclass[i]:
            my_counter_right += 1.0
    
    return my_counter_right/my_counter_total

In [5]:
# Model 2  y = a*SL*SW + b*PL*PW + c*SL + d*PL + e
def my_model_2() -> List[float]:
    # Define columns of products of variables
    df['SL SW'] = df['sepal-length']*df['sepal-width']
    df['PL PW'] = df['petal-length']*df['petal-width']
    # Put the 5 columns into an array
    olddata = np.asarray(df[['SL SW', 'PL PW', 'sepal-length', 'petal-length', 'ones']]).T
    
    ### Create the coefficient matrix A and load vector v, find the coef
    A = np.zeros((5,5))
    v = np.zeros((5,1))
    
    for i in range(5):
        v[i] = sum(olddata[i] * y.T[0])
        for j in range(5):
            A[i][j] = sum(olddata[i]*olddata[j])
    
    return np.matmul(inv(A),v)
    

In [6]:
# Classify one data record
def my_model_2_classify(newdata: List[float]) -> str:
    myindicator = sum(sum(mycoef.T*[newdata[0]*newdata[1], newdata[2]*newdata[3], newdata[0], newdata[2], 1]))
    
    if myindicator < 15:
        return "Iris-setosa"
    else:
        if myindicator < 25:
            return "Iris-versicolor"
        else:
            return "Iris-virginica"

In [7]:
# Calculate the accuracy 
def my_model_2_accuracy() -> float:
    my_counter_total = 0.0
    my_counter_right = 0.0
    datalist = np.asarray(df[['SL SW', 'PL PW', 'sepal-length', 'petal-length']])
    for i in range(len(oldclass)):
        my_counter_total += 1.0
        if my_model_2_classify(datalist[i]) == oldclass[i]:
            my_counter_right += 1.0
    
    return my_counter_right/my_counter_total

In [8]:
data = requests.get(
     "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
     )
with open('iris.dat', 'w') as f:
     f.write(data.text)

# Load the data into the data frame
df = pd.read_csv('iris.dat', header=None)

# Rename the columns
df.columns = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']

# Convert the classes (strings) into numbers
oldclass = np.asarray(df[['class']])
y = np.asarray(df[['sepal-length']])*0
for i in range(len(oldclass)):
    if oldclass[i] == 'Iris-setosa':
        y[i] = 10
    else:
        if oldclass[i] == 'Iris-versicolor':
            y[i] = 20
        else:
            y[i] = 30
# Insert the data field of numeric classes
df.insert(5,'new class', y, True)

In [9]:
mycoef = my_model_1()

# Classify the 10 inputs based on the regression model
print(my_model_1_classify([5.3,3.4,1.7,0.2]))
print(my_model_1_classify([5.4,3.7,1.5,0.2]))

# Get the accuracy rate
print(my_model_1_accuracy())


mycoef = my_model_2()

# Classify the 10 inputs based on the regression model
print(my_model_2_classify([5.3,3.4,1.7,0.2]))
print(my_model_2_classify([5.4,3.7,1.5,0.2]))

# Get the accuracy rate
print(my_model_2_accuracy())


Iris-setosa
Iris-setosa
0.9733333333333334
Iris-setosa
Iris-setosa
0.4066666666666667


In [11]:
print("------------ Model 1 -------------")
print(my_model_1_classify([5.3,3.4,1.7,0.2]))
print(my_model_1_classify([5.4,3.7,1.5,0.2]))
print(my_model_1_classify([5.5,2.6,4.4,1.2]))
print(my_model_1_classify([5.6,2.8,4.9,2.0]))
print(my_model_1_classify([5.7,2.5,5.0,2.0]))
print(my_model_1_classify([5.8,4.0,1.2,0.2]))
print(my_model_1_classify([5.9,3.0,4.2,1.5]))
print(my_model_1_classify([6.0,2.8,5.1,1.6]))
print(my_model_1_classify([6.1,2.5,5.6,1.4]))
print(my_model_1_classify([6.2,2.9,4.3,1.3]))


print("------------ Model 2 -------------")
print(my_model_2_classify([5.3,3.4,1.7,0.2]))
print(my_model_2_classify([5.4,3.7,1.5,0.2]))
print(my_model_2_classify([5.5,2.6,4.4,1.2]))
print(my_model_2_classify([5.6,2.8,4.9,2.0]))
print(my_model_2_classify([5.7,2.5,5.0,2.0]))
print(my_model_2_classify([5.8,4.0,1.2,0.2]))
print(my_model_2_classify([5.9,3.0,4.2,1.5]))
print(my_model_2_classify([6.0,2.8,5.1,1.6]))
print(my_model_2_classify([6.1,2.5,5.6,1.4]))
print(my_model_2_classify([6.2,2.9,4.3,1.3]))

------------ Model 1 -------------
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-versicolor
Iris-setosa
Iris-versicolor
------------ Model 2 -------------
Iris-setosa
Iris-setosa
Iris-versicolor
Iris-virginica
Iris-virginica
Iris-setosa
Iris-versicolor
Iris-virginica
Iris-virginica
Iris-versicolor
